In [1]:
# conda environment (project)
# /home/student/.conda/envs/project/bin/python 
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import os, gc
import torch

from transformers import set_seed
from datasets import load_dataset
from evaluate import *
from arc.arc import ARCSolver

from datasets import Dataset
from utils import render_grid

[2025-07-22 23:15:28,532] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


In [17]:
# prepare the test dataset
data_path = "dataset"
dataset, task_list = load_data(data_path)
df300 = sample_data(dataset, task_list, n_row=30000, random=12) 
df300.head(5) 

Imported 300 different tasks in the dataset.
# of samples in a task: min(82), Q1(1000), Q2(1000), Q3(1000), max(1000), mean(897.7)


,task,train,test_input,test_output,test
0,c59eb873,"[{'input': [[5, 5], [5, 5], [4, 5], [5, 5], [4...","[{'input': [[1, 8, 8, 9], [8, 6, 8, 6], [8, 8,...","[[[1, 1, 8, 8, 8, 8, 9, 9], [1, 1, 8, 8, 8, 8,...","[{'input': [[1, 8, 8, 9], [8, 6, 8, 6], [8, 8,..."
1,67a3c6ac,"[{'input': [[7, 7, 3], [2, 3, 1], [3, 3, 3], [...","[{'input': [[9, 9, 8, 7], [2, 7, 2, 9], [8, 9,...","[[[7, 8, 9, 9], [9, 2, 7, 2], [1, 9, 9, 8], [9...","[{'input': [[9, 9, 8, 7], [2, 7, 2, 9], [8, 9,..."
2,28bf18c6,"[{'input': [[9, 9, 9, 6, 9], [9, 6, 6, 9, 9], ...","[{'input': [[7, 7, 7, 7, 7, 7, 7, 7], [7, 7, 4...","[[[7, 4, 4, 4, 4, 7, 4, 4, 4, 4], [4, 7, 4, 4,...","[{'input': [[7, 7, 7, 7, 7, 7, 7, 7], [7, 7, 4..."
3,913fb3ed,"[{'input': [[5, 5, 5, 5, 5, 5, 5, 5, 5, 5], [5...","[{'input': [[0, 0, 0, 0, 2, 0, 0, 0, 0, 8], [0...","[[[6, 6, 6, 1, 2, 1, 0, 0, 4, 8], [6, 3, 6, 1,...","[{'input': [[0, 0, 0, 0, 2, 0, 0, 0, 0, 8], [0..."
4,623ea044,"[{'input': [[8, 8, 8, 8, 8, 8], [8, 8, 8, 8, 8...","[{'input': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0,...","[[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0...","[{'input': [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0,..."


In [3]:
# prepare samples for each task
task_samples = []
for t in range(300):
    df = sample_data(dataset, task_list, n_row=1000, indices=[t])
    task_samples.append(df)

In [4]:
# Visualize a task (EDA)
task_indices = [1, 6, 8, 10, 13, 15, 17] # select which task you want to examine
n_sample = 1
for task_idx in task_indices:
    print(task_idx)
    for data in Dataset.from_pandas(task_samples[task_idx]).shuffle().select(range(n_sample)):
        for case in data['train']:
            print("==================================================")
            print("Example input")
            render_grid(case['input'])
            print("Example output")
            render_grid(case['output'])
            break
        print("==================================================")
        print("Example test input")
        render_grid(data['test'][0]['input'])
        print("Example test output")
        render_grid(data['test'][0]['output'])
    print("==================================================")

1
Example input


                [5, 7, 7, 7, 7, 7, 5]
                [7, 7, 7, 7, 7, 7, 7]
                [7, 7, 7, 7, 7, 7, 7]
                [7, 7, 7, 7, 7, 7, 7]
                [5, 7, 7, 7, 7, 7, 5]

Example output


                [5, 8, 5, 8, 5, 8, 5]
                [8, 5, 8, 5, 8, 5, 8]
                [5, 8, 5, 8, 5, 8, 5]
                [8, 5, 8, 5, 8, 5, 8]
                [5, 8, 5, 8, 5, 8, 5]

Example test input


...

        [8, 7, 2]
        [2, 9, 7]

Example test input


                      [2, 7, 2, 2, 2, 2, 2, 2, 7, 2]
                      [7, 8, 2, 1, 2, 2, 1, 2, 8, 7]
                      [2, 2, 0, 2, 7, 7, 2, 0, 2, 2]
                      [2, 1, 2, 0, 2, 2, 0, 2, 1, 2]
                      [2, 2, 7, 2, 2, 2, 2, 7, 2, 2]
                      [2, 2, 7, 2, 2, 2, 2, 7, 2, 2]
                      [2, 1, 2, 0, 2, 2, 0, 2, 1, 2]
                      [2, 2, 0, 2, 7, 7, 2, 0, 2, 2]
                      [3, 8, 2, 1, 2, 2, 1, 2, 8, 7]
                      [2, 7, 2, 2, 2, 2, 2, 2, 7, 2]

Example test output


    [7]

In [5]:
simple_tasks = []
hard_tasks = []
for task_idx in range(300):
    check = True
    for data in Dataset.from_pandas(task_samples[task_idx]).shuffle().select(range(3)):
        for case in data['train']:
            wi, hi = len(case['input'][0]), len(case['input'])
            wo, ho = len(case['output'][0]), len(case['output'])
            if (wi!=wo) or (hi!=ho): check = False
        case = data['test'][0]
        wi, hi = len(case['input'][0]), len(case['input'])
        wo, ho = len(case['output'][0]), len(case['output'])
        if (wi!=wo) or (hi!=ho): check = False
    if check: simple_tasks.append(task_idx)
    else: hard_tasks.append(task_idx)
print(simple_tasks)
dfsimple = sample_data(dataset, task_list, n_row=n_train+n_eval, indices=simple_tasks, random=56)
dfhard = sample_data(dataset, task_list, n_row=n_train+n_eval, indices=hard_tasks, random=56)

[0, 1, 3, 7, 8, 9, 10, 11, 13, 15, 18, 19, 20, 22, 27, 29, 30, 32, 33, 34, 35, 38, 40, 41, 44, 45, 46, 51, 52, 54, 55, 56, 57, 58, 61, 64, 65, 66, 67, 68, 69, 70, 72, 73, 75, 78, 79, 80, 81, 83, 85, 87, 89, 90, 91, 92, 93, 95, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 110, 111, 113, 114, 116, 117, 118, 119, 120, 123, 124, 128, 129, 130, 133, 135, 137, 138, 139, 140, 141, 142, 143, 145, 147, 148, 150, 151, 153, 154, 155, 157, 158, 159, 160, 161, 162, 166, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 181, 182, 185, 188, 190, 193, 194, 195, 196, 197, 198, 201, 203, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 218, 220, 222, 224, 225, 226, 227, 228, 230, 231, 233, 234, 235, 236, 239, 244, 245, 246, 248, 249, 250, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 264, 267, 268, 270, 271, 273, 274, 275, 276, 277, 281, 282, 285, 286, 287, 291, 292, 293, 295, 296, 297]


In [54]:
set_seed(1234567890)
token = os.environ.get("HF_TOKEN", None)
solver = ARCSolver(model_id="Qwen/Qwen3-1.7B", hf_token=token)

In [75]:
# before training, wandb.login() at python console. 
# always run this block before re-training. 
import wandb
wandb.finish()

profiling/Time taken: GRPOTrainer._get_per_token_logps,▆▁▁▁▆▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁█▁
profiling/Time taken: GRPOTrainer._prepare_inputs,▁▁▁▄▁▃▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁
profiling/Time taken: GRPOTrainer.compute_loss,▅▁▁▃▃▂▅▄▇▄▂▂▅▂▄▄▂▁▄▇█▂▅▂▁▂▄▂▄▄▁▃▃▆▂▃▂▆▃▂
profiling/Time taken: GRPOTrainer.reward,▂▃▅▄▁▅▅▃▅▃▂▇▂▂▃▅▃▆▃▃▃▅▂▃█▃▂▄▄▄▂▂▂
train/clip_ratio/high_max,▁▁▁
train/clip_ratio/high_mean,▁▁▁
train/clip_ratio/low_mean,▁▁▁
train/clip_ratio/low_min,▁▁▁
train/clip_ratio/region_mean,▁▁▁
train/completions/clipped_ratio,█▁▁
train/completions/max_length,▄▁█


In [76]:
n_train = 30000
n_eval = 500
train_dataset = Dataset.from_pandas(df300).select(range(n_train))
# solver.prepare_train()
# solver.train(train_dataset)
solver.prepare_train()
solver.train_rlvr(train_dataset, checkpoint="20250709_120108/checkpoint-final")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

After truncating, there are 29681 rows left in the dataset.


No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 69,730,304 || all params: 1,790,305,280 || trainable%: 3.8949


Step,Training Loss
10,-0.004700
20,-0.006600
30,0.013400
40,0.007000
50,-0.000400
60,0.006600
70,0.003600
80,0.004400
90,0.000700
100,0.004100


: 

In [ ]:
n_eval = 100
solver.prepare_evaluation(select_adapter="20250709_120108") # make sure you set the right model

In [ ]:
# evaluate our model (eval set)
scores = []
scores_max = []
n_eval = 20
ttt = True
task_indices = [6,8,9,10,13,14,15,17] 
scores_task = []
scores_max_task = []
for task in task_indices:
    eval_dataset = Dataset.from_pandas(task_samples[task]).select(range(n_eval))

    for eval_data in tqdm(eval_dataset):
        print("============================================")
        print("Test input")
        render_grid(eval_data["test"][0]['input'])

        print("Predict output")
        if ttt: solver.train_testtime(eval_data) # TTT
        preds, candidate_scores = [], []
        for _ in range(10):
            p, cs = solver.predict(eval_data, use_ttt=ttt, return_score=True)
            dup = -1
            for i, q in enumerate(preds):
                if p.shape == q.shape and (p == q).all(): 
                    cs = np.log(np.exp(cs)+np.exp(candidate_scores[i]))
                    dup = i; break 
            if dup == -1: 
                preds.append(p)
                candidate_scores.append(cs)
            else: candidate_scores[dup] = cs
            # render_grid(p)

        s, s_max, cs_max, i_max = 0, 0, float('-inf'), -1
        for i, p in enumerate(preds):
            if p is None: continue
            check = check_match(p, eval_data["test"][0]["output"])
            s_max = max(s_max, check)
            cs = candidate_scores[i]
            if cs > cs_max: 
                cs_max = cs; s = check; i_max = i
            if check: print("Here! vv")
            print(cs)
        scores.append(s)
        scores_max.append(s_max)
        print("Selected candidate")
        render_grid(preds[i_max])

        print("Test output")
        render_grid(eval_data["test"][0]['output'])
        print("============================================")

    score = np.array(scores).mean() * 100
    score_max = np.array(scores_max).mean() * 100
    scores_task.append(score)
    scores_max_task.append(score_max)
    print(f"Evaluation score: {score:.2f}", flush=True)
    print(f"Max Evaluation Score: {score_max:.2f}", flush=True)
    scores = []
    scores_max = []

In [ ]:
from matplotlib import pyplot as plt
x = np.arange(len(scores_task))
width = 0.4
plt.bar(x-width/2, scores_max_task, width, label='scores_max_task')
plt.bar(x+width/2, scores_task, width, label='scores_task')
plt.xticks(x, task_indices)
plt.ylim(0,100)
plt.show()